In [1]:
import os
import sqlite3
import pandas as pd

In [2]:
def creer_table_si_absente(cursor, nom_table, colonnes):
    """
    Crée une table dans la base de données si elle n'existe pas déjà,
    en utilisant les colonnes extraites de l'en-tête du fichier CSV.
    """
    colonnes_avec_types = ', '.join([f"{colonne} TEXT" for colonne in colonnes])
    requete_creation = f"CREATE TABLE IF NOT EXISTS {nom_table} ({colonnes_avec_types});"
    cursor.execute(requete_creation)


In [4]:
def inserer_donnees(cursor, nom_table, colonnes, donnees):
    """
    Insère les données dans la table. 
    """
    placeholders = ', '.join(['?' for _ in colonnes])
    colonnes_str = ', '.join(colonnes)
    requete_insertion = f"INSERT INTO {nom_table} ({colonnes_str}) VALUES ({placeholders});"
    cursor.executemany(requete_insertion, donnees)

In [12]:
def traiter_fichier(chemin_fichier, connection, nom_table="personnes"):
    """
    Traite un fichier CSV en créant une table correspondante et en y insérant les données.
    """
    #nom_table = os.path.splitext(os.path.basename(chemin_fichier))[0]  # Utilise le nom du fichier pour la table
    df = pd.read_csv(chemin_fichier,sep=';')  # Charge le fichier CSV avec pandas
    colonnes = list(df.columns)

    cursor = connection.cursor()

    # Créer la table si elle n'existe pas
    creer_table_si_absente(cursor, nom_table, colonnes)

    # Insérer les données dans la table
    inserer_donnees(cursor, nom_table, colonnes, df.values.tolist())

    # Commit les changements
    connection.commit()

    print(f"Traitement du fichier : {chemin_fichier} terminé.")

In [13]:
def parcourir_arborescence(chemin_racine, db_path):
    """
    Parcourt récursivement l'arborescence et traite chaque fichier CSV trouvé.
    """
    # Créer ou se connecter à la base de données
    connection = sqlite3.connect(db_path)

    print(f"Parcours de l'arborescence à partir de : {chemin_racine}")

    for racine, sous_repertoires, fichiers in os.walk(chemin_racine):
        print(f"Répertoire courant : {racine}")
        for sous_repertoire in sous_repertoires:
            print(f"Répertoire trouvé : {sous_repertoire}")
        for fichier in fichiers:
            if fichier.endswith(".csv"):
                chemin_fichier = os.path.join(racine, fichier)
                traiter_fichier(chemin_fichier, connection)

    # Fermer la connexion à la base de données
    connection.close()

In [14]:

chemin_racine = "C:/Users/michel/Documents/ICAM/files" 
db_path = "data.db" 
parcourir_arborescence(chemin_racine, db_path)


Parcours de l'arborescence à partir de : C:/Users/michel/Documents/ICAM/files
Répertoire courant : C:/Users/michel/Documents/ICAM/files
Répertoire trouvé : détail_files
Répertoire courant : C:/Users/michel/Documents/ICAM/files\détail_files
Traitement du fichier : C:/Users/michel/Documents/ICAM/files\détail_files\file1.csv terminé.
Traitement du fichier : C:/Users/michel/Documents/ICAM/files\détail_files\file2.csv terminé.
